In [1]:
# 参考文献

# 線分の交差判定
# https://qiita.com/wihan23/items/03efd7cd40dfec96a987

# 焼きなましについて
# https://shindannin.hatenadiary.com/entry/2021/03/06/115415

In [2]:
#!/usr/bin/env python3

import sys
import math
import random
import time
import csv
import copy


In [3]:
# calculate a probability for simulated annealing
def getProbability(start_time, time_limit, new_total_distance, total_distance, initial_distance):
    if (new_total_distance<total_distance):
        return 1.0
    start_temp = initial_distance*0.025;
    end_temp = initial_distance*0.0075;
    temp = start_temp + (end_temp - start_temp) * (time.time()/(start_time+time_limit))
    probability = math.exp((total_distance-new_total_distance)/temp)
    return probability

In [4]:
# calculate a distance between the two points
def distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)

In [5]:
# swap two points inside the route
def swapRoute(cities, tour, total_distance, first, second):
    
    new_tour = tour[:]
    new_total_distance = total_distance
    
    subtract1 = distance (cities[new_tour[first]], cities[new_tour[first+1]])
    subtract2 = distance (cities[new_tour[second]], cities[new_tour[second+1]])
    subtract3 = distance (cities[new_tour[first]], cities[new_tour[first-1]])
    subtract4 = distance (cities[new_tour[second]], cities[new_tour[second-1]])
    
    temp = new_tour[first]
    new_tour[first] = new_tour[second]
    new_tour[second] = temp
    
    add1 = distance (cities[new_tour[first]], cities[new_tour[first+1]])
    add2 = distance (cities[new_tour[second]], cities[new_tour[second+1]])
    add3 = distance (cities[new_tour[first]], cities[new_tour[first-1]])
    add4 = distance (cities[new_tour[second]], cities[new_tour[second-1]])
    
    new_total_distance -= (subtract1 + subtract2 + subtract3 + subtract4)
    new_total_distance += (add1 + add2 + add3 + add4)
    
    return new_tour, new_total_distance

In [6]:
# checking whether x or y coordinates of two lines (created by p1&p2 and p3&p4) has an intersection
def max_min_cross(p1, p2, p3, p4):
    min_ab = min(p1, p2)
    max_ab = max(p1, p2)
    min_cd = min(p3, p4)
    max_cd = max(p3, p4)

    if min_ab > max_cd or max_ab < min_cd:
        return False

    return True


# checking whether two lines (created by p1&p2 and p3&p4 has an intersection)
def judgeCross(i, j, cities, tour):
    first = cities[tour[i]]
    second = cities[tour[j]]
    first_next = cities[tour[i+1]]
    second_next = cities[tour[j+1]]
    # x座標による判定
    if not max_min_cross(first[0], first_next[0], second[0], second_next[0]):
        return False

    # y座標による判定
    if not max_min_cross(first[1], first_next[1], second[1], second_next[1]):
        return False

    tc1 = (first[0] - first_next[0]) * (second[1] - first[1]) + (first[1] - first_next[1]) * (first[0] - second[0])
    tc2 = (first[0] - first_next[0]) * (second_next[1] - first[1]) + (first[1] - first_next[1]) * (first[0] - second_next[0])
    td1 = (second[0] - second_next[0]) * (first[1] - second[1]) + (second[1] - second_next[1]) * (second[0] - first[0])
    td2 = (second[0] - second_next[0]) * (first_next[1] - second[1]) + (second[1] - second_next[1]) * (second[0] - first_next[0])
    return tc1 * tc2 <= 0 and td1 * td2 <= 0

In [7]:
# when the two lines intersect each other, reorganize the points so that there is no intersections
def noCross(cities, tour, total_distance):
    continue_flag = True
    while continue_flag:
        continue_flag = False
        N = len(cities)
        for i in range (1, N-2):
            for j in range(i+2, N-1):
                if judgeCross(i, j, cities, tour):
                    #from IPython.core.debugger import Pdb; Pdb().set_trace()
                    tour, temp_distance = fixChain(cities, tour, total_distance, i, j)
                    if temp_distance<total_distance:
                        continue_flag = True
                        total_distance = temp_distance
    
    return tour, total_distance

In [8]:
# changing the order of the tour and re-calculate the distance of the given two points
def fixChain(cities, tour, total_distance, start, end):
    new_tour = tour[:]
    new_total_distance = 0
    for i in range (end-start):
        new_tour[start+i+1]=tour[end-i]
        
    N = len(cities)
    
    for i in range (N-1):
        new_total_distance += distance(cities[new_tour[i]], cities[new_tour[i+1]])
        
    return new_tour, new_total_distance

In [9]:
# simulated annealing
def annealing(cities, tour, total_distance, initial_distance, start_time, time_limit):
    first = random.randrange(len(tour))
    second = random.randrange(len(tour))
    
    if first==0 or second ==0:
        return tour, total_distance
    if first==len(tour)-1 or second==len(tour)-1:
        return tour, total_distance
    if first==second:
        return tour, total_distance
    
    new_tour, new_total_distance = swapRoute(cities, tour, total_distance, first, second) #TODO
    
    rand = random.random()
    probability = getProbability(start_time, time_limit, new_total_distance, total_distance, initial_distance)
    
    print(str(probability)+" , "+str(rand)+" , "+str(new_total_distance)+" , "+str(total_distance))
    
#     Annealing1
    if probability>rand:
        tour = new_tour
        total_distance = new_total_distance

#   Annealing2
#     if new_total_distance < total_distance:
#         tour = new_tour
#         total_distance = new_total_distance
    
    return tour, total_distance

In [10]:
# creating an initial answer based on greedy technique
def greedy(cities):
    N = len(cities)

    dist = [[0] * N for i in range(N)]
    for i in range(N):
        for j in range(i, N):
            dist[i][j] = dist[j][i] = distance(cities[i], cities[j])

    current_city = 0
    unvisited_cities = set(range(1, N))
    tour = [current_city]
    total_distance = 0
    
    while unvisited_cities:
        next_city = min(unvisited_cities,
                        key=lambda city: dist[current_city][city])
        total_distance += dist[current_city][next_city]
        unvisited_cities.remove(next_city)
        tour.append(next_city)
        current_city = next_city
        
        
    total_distance += dist[current_city][tour[0]]
    tour.append(tour[0])
    return tour, total_distance

In [11]:
# do brute-force method if the number of cities is small
def brute(cities, tour, total_distance):
    if len(cities)<9:
        now_place = 0
        visited = [False] * len(cities)
        visited[0]=True
        count_visited = 1
        now_distance = 0
        now_tour = [0]
        tour, total_distance = run_brute(cities, tour, now_tour, now_distance, now_place, visited, count_visited, total_distance)
    
    return tour, total_distance


def run_brute (cities, tour, now_tour, now_distance, now_place, visited, count_visited, min_distance):
    if count_visited == len(cities):
        now_distance += distance(cities[now_place], cities[0])
        now_tour.append(0)
        if now_distance<min_distance:
            tour = now_tour
            min_distance = now_distance
        return tour, min_distance
        
    for i in range (len(cities)):
        if visited[i]:
            continue
            
        visited[i]=True
        next_tour = now_tour[:]
        next_tour.append(i)
        next_distance = now_distance + distance(cities[now_place], cities[i])
        tour, min_distance = run_brute (cities, tour, next_tour, next_distance, i, visited, count_visited+1, min_distance)
        visited[i]=False
        
    return tour, min_distance

In [12]:
# method for creating an initial answer and calling functions to improve it
def solve(cities):
    tour, total_distance = greedy(cities)
    start_time = time.time()
    time_limit = 1
    initial_distance = total_distance
    while (time.time()-start_time<time_limit):
        tour, total_distance = annealing(cities, tour, total_distance, initial_distance, start_time, time_limit)
    tour, total_distance = noCross(cities, tour, total_distance)
    tour, total_distance = brute(cities, tour, total_distance)
    #from IPython.core.debugger import Pdb; Pdb().set_trace()
    return tour

In [13]:
# reading input from csv file
def readInput(count):
    with open("input_"+str(count)+".csv", encoding='utf-8') as csvfile:
        cities = []
        csvreader = csv.reader(csvfile)
        next(csvreader)  # Skip the header row
        for row in csvreader:
            x, y = float(row[0]), float(row[1])
            cities.append([x, y])
    
    return cities

In [14]:
# writing the answer as a csv file
def writeOutput(tour, count):
    
    tour = tour[:-1]
    
    with open('output_'+str(count)+".csv", 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Write the header
        csvwriter.writerow(['index'])
        # Write the tour indices
        for location in tour:
            csvwriter.writerow([location])

In [15]:
if __name__ == '__main__':
#     for i in range (7):
#         cities = readInput(i)
#         tour = solve(cities)
#         writeOutput(tour, i)
#         print("Finished "+str(i)+"th data!")

# When testing for one particular test case
    i = 5
    cities = readInput(i)
    tour = solve(cities)
    writeOutput(tour, i)
    print("Finished "+str(i)+"th data!")

1.0 , 0.8934264309925561 , 24064.763689481184 , 25331.843307461648
0.43450802495745766 , 0.15969947074290392 , 24223.1271390727 , 24064.763689481184
1.0 , 0.6470565787557795 , 24223.1271390727 , 24223.1271390727
1.0 , 0.47318562986392043 , 24223.1271390727 , 24223.1271390727
0.000166879138804288 , 0.7190101149591617 , 25875.69567497016 , 24223.1271390727
0.027776465829951975 , 0.11236558010913589 , 24903.964665188858 , 24223.1271390727
0.5606951968278452 , 0.8796160993533019 , 24333.050463533993 , 24223.1271390727
0.15492897356160104 , 0.017117385225133508 , 24577.41611559151 , 24223.1271390727
0.04233784678276811 , 0.028369947428528497 , 25178.174814907183 , 24577.41611559151
0.14397956667463147 , 0.6811649935636074 , 25546.389095579496 , 25178.174814907183
0.0006821563311711855 , 0.88313510787487 , 26563.241170142755 , 25178.174814907183
5.690314208728535e-05 , 0.06803795842292226 , 27035.155988738057 , 25178.174814907183
1.0 , 0.1950480524083965 , 25178.174814907183 , 25178.17481490

0.0016224641673452467 , 0.09867624115211748 , 32433.533411053733 , 31213.081450158275
0.00045381113041201954 , 0.41511368004956684 , 32675.583024095333 , 31213.081450158275
1.0 , 0.8833695604452513 , 31213.081450158275 , 31213.081450158275
0.14691903922435987 , 0.5159777931661246 , 31577.456001448074 , 31213.081450158275
0.26076675580280956 , 0.2668804157255157 , 31468.45092567496 , 31213.081450158275
5.301782313446186e-05 , 0.5253101586345137 , 33083.499092532824 , 31213.081450158275
0.00017053744318536515 , 0.8616623032694491 , 32861.53006833196 , 31213.081450158275
1.0 , 0.44008541878156027 , 31213.081450158275 , 31213.081450158275
1.0 , 0.8484625839796658 , 31213.081450158275 , 31213.081450158275
0.0009625207778017381 , 0.941791641975068 , 32532.735261013502 , 31213.081450158275
3.282454336181983e-06 , 0.9105844631604232 , 33612.05498135574 , 31213.081450158275
1.0 , 0.3703813824051996 , 31213.081450158275 , 31213.081450158275
1.0 , 0.5056720411096481 , 31213.081450158275 , 31213.0

1.5062295825222135e-05 , 0.43161243421162254 , 37768.52982306051 , 35659.02388188479
1.0 , 0.8030984637349114 , 35659.02388188479 , 35659.02388188479
1.0 , 0.27784294320567204 , 35659.02388188479 , 35659.02388188479
0.004779896995034523 , 0.0446764599506454 , 36674.19806334489 , 35659.02388188479
1.0 , 0.18717004250229252 , 35659.02388188479 , 35659.02388188479
1.0 , 0.769446853197169 , 35659.02388188479 , 35659.02388188479
0.02971685740506185 , 0.35936004088970763 , 36327.03234283391 , 35659.02388188479
1.8020514243872764e-06 , 0.4733817678219576 , 38171.927197764 , 35659.02388188479
1.0 , 0.5591391992944417 , 35659.02388188479 , 35659.02388188479
0.0013816742800905723 , 0.5171370428886025 , 36909.99756122924 , 35659.02388188479
0.00011573138675628072 , 0.5436016571856086 , 37381.12793834715 , 35659.02388188479
0.0002549731963290059 , 0.26280070425717983 , 37231.05832035617 , 35659.02388188479
1.0 , 0.885675284584669 , 35659.02388188479 , 35659.02388188479
1.0 , 0.3378446604580799 , 3

0.2555992544563399 , 0.6042451516572881 , 41997.25838893358 , 41738.086183483945
0.011546654527456077 , 0.606954096356995 , 42585.69463894657 , 41738.086183483945
1.0 , 0.14339806577500158 , 41738.086183483945 , 41738.086183483945
0.08653164960700678 , 0.3538611034940847 , 42203.03539292637 , 41738.086183483945
1.0 , 0.6306286321192894 , 41738.086183483945 , 41738.086183483945
0.01149895047765062 , 0.8909515407372842 , 42586.48118799078 , 41738.086183483945
0.30828398052065614 , 0.25778750432340336 , 41961.6524755019 , 41738.086183483945
1.0 , 0.80627815547069 , 41961.6524755019 , 41961.6524755019
1.0 , 0.22199911101998238 , 41961.6524755019 , 41961.6524755019
4.406378380564146e-05 , 0.6068897366081645 , 43867.21613538664 , 41961.6524755019
1.0 , 0.428911244794156 , 41961.6524755019 , 41961.6524755019
1.1825490213892514e-05 , 0.47590637451080664 , 44117.12379295681 , 41961.6524755019
0.02788064167841522 , 0.24812281981055917 , 42641.77877929175 , 41961.6524755019
1.0 , 0.50820404784351

0.028214875728183986 , 0.14777815557018004 , 50037.85799756164 , 49359.99574247786
0.0025408821435677098 , 0.3641581656964997 , 50495.225319982426 , 49359.99574247786
1.0 , 0.6348585783722978 , 49359.99574247786 , 49359.99574247786
0.005693355589657999 , 0.39571548361451203 , 50341.944523360835 , 49359.99574247786
1.0 , 0.32386269752638397 , 49359.99574247786 , 49359.99574247786
1.0 , 0.5399433065828657 , 49359.99574247786 , 49359.99574247786
0.0021789798496501217 , 0.5224347395129563 , 50524.417960321654 , 49359.99574247786
1.0 , 0.02516456310330273 , 49359.99574247786 , 49359.99574247786
1.0 , 0.9864168858037882 , 49359.99574247786 , 49359.99574247786
0.004566810485294508 , 0.905108578092928 , 50383.83416609298 , 49359.99574247786
1.0 , 0.8644979678388764 , 49359.99574247786 , 49359.99574247786
1.0 , 0.26686967036091014 , 49359.99574247786 , 49359.99574247786
1.0 , 0.4309671107394095 , 49359.99574247786 , 49359.99574247786
0.8703940221496282 , 0.8550430437229773 , 49386.36795271803 ,

Finished 5th data!
